In [1]:

# ✅ Instalación mínima
!pip install -q opencv-python-headless tensorflow


In [2]:

import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import layers, models, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split


In [3]:

# ✅ Rutas de imágenes y anotaciones YOLO
img_dir = "/content/drive/MyDrive/yolov8/images/train/images"
lbl_dir = "/content/drive/MyDrive/yolov8/images/train/labels"


In [7]:
# ✅ Leer imágenes y anotaciones para regresión de bounding boxes
def load_detection_data(img_dir, lbl_dir, target_size=(128, 128), max_imgs=1000):
    X, Y = [], []  # Usamos Y en lugar de y para evitar conflicto de nombre
    for fname in os.listdir(lbl_dir)[:max_imgs]:
        if not fname.endswith('.txt'):
            continue
        img_name = fname.replace('.txt', '.jpg')
        img_path = os.path.join(img_dir, img_name)
        lbl_path = os.path.join(lbl_dir, fname)

        if not os.path.exists(img_path):
            continue

        img = cv2.imread(img_path)
        if img is None:
            continue

        h0, w0 = img.shape[:2]
        img_resized = cv2.resize(img, target_size)
        img_resized = img_resized / 255.0

        with open(lbl_path, 'r') as f:
            line = f.readline().strip().split()
            if len(line) != 5:
                continue  # clase, x, y, w, h
            _, cx, cy, w, h = map(float, line)  # Renombramos para evitar conflicto
            box = [cx, cy, w, h]

        X.append(img_resized)
        Y.append(box)

    return np.array(X), np.array(Y)

# Usamos variables finales X, y (que no entran en conflicto dentro de la función)
X, y = load_detection_data(img_dir, lbl_dir)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
print("Datos cargados:", X_train.shape)


Datos cargados: (800, 128, 128, 3)


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

def build_object_detector():
    inputs = Input(shape=(128, 128, 3))
    x = layers.Conv2D(32, 3, activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(64, 3, activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(4, activation='sigmoid')(x)  # x, y, w, h
    return models.Model(inputs, outputs)

detector = build_object_detector()
detector.compile(optimizer='adam', loss='mse')


In [ ]:

detector.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


In [ ]:

def create_custom_classifier(num_classes):
    inputs = Input(shape=(128, 128, 3))
    x = layers.Conv2D(16, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D()(x)
    x = layers.Conv2D(32, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D()(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(64, activation='relu')(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)
    return models.Model(inputs, outputs)


In [ ]:

# Supongamos que ya tenemos un modelo de clasificación entrenado
# cnn_model = create_custom_classifier(num_classes=12)
# cnn_model.load_weights('/content/drive/MyDrive/Model_Checkpoints/best_model.h5')

def detectar_y_clasificar_custom(img_path, detector, classifier):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (128, 128)) / 255.0
    input_array = np.expand_dims(img_resized, axis=0)

    pred_box = detector.predict(input_array)[0]
    h, w = img.shape[:2]

    cx, cy, bw, bh = pred_box
    x1 = int((cx - bw / 2) * w)
    y1 = int((cy - bh / 2) * h)
    x2 = int((cx + bw / 2) * w)
    y2 = int((cy + bh / 2) * h)

    crop = img_rgb[max(0,y1):min(h,y2), max(0,x1):min(w,x2)]
    if crop.size == 0:
        print("Recorte inválido.")
        return
    crop_resized = cv2.resize(crop, (128, 128)) / 255.0
    crop_input = np.expand_dims(crop_resized, axis=0)

    pred = classifier.predict(crop_input)
    class_id = np.argmax(pred)
    confidence = np.max(pred)
    print(f"Clase: {class_id}, Confianza: {confidence:.2f}")
